In [1]:
import sys

import keras_tuner as kt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers

from src.cross_validator import KerasTunerCrossValidator
from src.dataset import load_dataset_train_test, load_dataset_train_val
from src.models import rnn_builder as bare_model_builder
from src.network_utils import count_params

PWD = '../../..'
sys.path.append(PWD)
tf.get_logger().setLevel('ERROR')

In [2]:
PROJECT_NAME = 'rnn'

PLANE = 2
CHANNEL = 11

OVERWRITE = False

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/cross_val'

LR = 0.01
ES_MIN_DELTA = 0.01

N_EPOCHS = 3000
BATCH_SIZE = 4096
MAX_TRIALS = 15
EXECUTIONS_PER_TRIAL = 2

TOP_N = 4
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_base_train, _, y_base_train, _ = load_dataset_train_test(PWD, PLANE, CHANNEL)
X_train, X_val, y_train, y_val = load_dataset_train_val(PWD, PLANE, CHANNEL)

X_base_train.shape, X_train.shape, X_val.shape

((22134, 24), (17707, 24), (4427, 24))

# Model

In [4]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    hp_rnn_type = hp.Choice("rnn_type", values=["lstm", "gru"])
    hp_n_neurons = hp.Choice("n_neurons", values=[16, 32, 64, 128, 256, 512])
    hp_n_hidden_layers = hp.Int("n_hidden_layers", min_value=0, max_value=1, default=0)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization")
    hp_normalize_signal = hp.Boolean("normalize_signal", default=False)

    model = bare_model_builder(hp_rnn_type, hp_n_neurons, hp_n_hidden_layers, hp_input_batch_normalization,
                               hp_normalize_signal)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [5]:
model_builder(kt.HyperParameters()).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 24, 1)             0         
                                                                 
 lstm (LSTM)                 (None, 16)                1152      
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 1,169
Trainable params: 1,169
Non-trainable params: 0
_________________________________________________________________


In [6]:
model_callbacks = [
    callbacks.EarlyStopping(patience=60, min_delta=ES_MIN_DELTA),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=10)
]

# Bayesian tuner

In [7]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR, project_name=PROJECT_NAME,
                                         overwrite=OVERWRITE)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

# Results

In [8]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'rnn_type': 'lstm', 'n_neurons': 128, 'n_hidden_layers': 1, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 198369
========== Model 1 ==========
{'rnn_type': 'gru', 'n_neurons': 128, 'n_hidden_layers': 1, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 149601
========== Model 2 ==========
{'rnn_type': 'gru', 'n_neurons': 64, 'n_hidden_layers': 0, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 13025
========== Model 3 ==========
{'rnn_type': 'gru', 'n_neurons': 16, 'n_hidden_layers': 0, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 1025


# Cross-validation for top 5 models

In [9]:
cross_validator = KerasTunerCrossValidator(bayesian_tuner, X_base_train, y_base_train, model_builder,
                                           directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                           n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                           es_min_delta=ES_MIN_DELTA, n_cv=CROSSVAL_N_CV, n_executions=CROSSVAL_N_EXEC,
                                           overwrite=OVERWRITE)
model_scores = cross_validator()

{'rnn_type': 'lstm', 'n_neurons': 128, 'n_hidden_layers': 1, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 198369
Got score: 8.7285 (8.7522, 8.7047)
Got score: 8.1632 (8.1685, 8.1579)
Got score: 8.4030 (8.3476, 8.4584)
Got score: 8.3626 (8.3705, 8.3548)
Got score: 8.2031 (8.1709, 8.2353)


{'rnn_type': 'gru', 'n_neurons': 128, 'n_hidden_layers': 1, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 149601
Got score: 8.7600 (8.7675, 8.7526)
Got score: 8.1460 (8.1371, 8.1550)
Got score: 8.3453 (8.3189, 8.3717)
Got score: 8.3786 (8.3836, 8.3736)
Got score: 8.1893 (8.2032, 8.1755)


{'rnn_type': 'gru', 'n_neurons': 64, 'n_hidden_layers': 0, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 13025
Got score: 8.7908 (8.8093, 8.7724)
Got score: 8.1429 (8.1416, 8.1442)
Got score: 8.3646 (8.3903, 8.3389)
Got score: 8.4953 (8.4388, 8.5519)
Got score: 8.2509 (8.2322, 8.2695)


{'rnn_type': 'gru', 'n_neurons': 16, 'n_hidden_layers': 0, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 1025
Got score: 8.8141 (8.8328, 8.7954)
Got score: 8.1565 (8.1655, 8.1476)
Got score: 8.3641 (8.3514, 8.3767)
Got score: 8.3996 (8.4143, 8.3849)
Got score: 8.1741 (8.1719, 8.1763)


In [10]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,8.37,0.20,198369
1,8.36,0.22,149601
2,8.41,0.22,13025
3,8.38,0.24,1025
